In [6]:
from tavily import TavilyClient
from deepagents import create_deep_agent
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def internet_search(query: str, max_results: int = 5):
    """Run a web search"""
    return tavily_client.search(query, max_results=max_results)

# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher. Your job is to conduct thorough research and then write a polished report.

You have access to an internet search tool as your primary means of gathering information.

## `internet_search`

Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.
"""

agent = create_deep_agent(
    tools=[internet_search],
    system_prompt=research_instructions,
    model=llm,
)



result = agent.invoke({"messages": [{"role": "user", "content": "What is langchain deep agent? "}]})

result


{'messages': [HumanMessage(content='What is langchain deep agent? ', additional_kwargs={}, response_metadata={}, id='30e06ecf-6671-4de1-80b8-425aeb9193ea'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'internet_search', 'arguments': '{"query": "langchain deep agent"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--70894cc7-6d3d-4455-a78c-a0e1eec5b987-0', tool_calls=[{'name': 'internet_search', 'args': {'query': 'langchain deep agent'}, 'id': 'd9b1cfd3-0fdd-4325-bbca-cfcb043d7ca4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 5070, 'output_tokens': 78, 'total_tokens': 5148, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 60}}),
  ToolMessage(content='{"query": "langchain deep agent", "follow_up_questions": null, "answer": null, "imag

In [2]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--d0e9abe7-8e70-4746-99ec-114fad3d1ece-0', usage_metadata={'input_tokens': 21, 'output_tokens': 7, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})

In [8]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
import os
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("code_generator_v1", include_model=True)

prompt

PromptTemplate(input_variables=['architecture', 'constraints_optional', 'existing_files_optional', 'project_context', 'requirements', 'technology_stack'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'code_generator_v1', 'lc_hub_commit_hash': '63cb9932239e6d8d19dd545da7e36ef053ca64ff01e8e43a095c2c19936fd223'}, template='SYSTEM ROLE\nYou are the CodeGen Agent in a multi-agent swarm. Your job is to generate production-ready code strictly from the inputs provided. Produce outputs that downstream tools can execute, test, and review without manual edits.\n\nOPERATING PRINCIPLES\n- Grounding: Use ONLY the provided context. If something is missing, state concise ASSUMPTIONS (must be reasonable and minimal).\n- Determinism: Avoid vague placeholders and "TODOs". Use stable, pinned dependency versions where applicable.\n- Security: No hardcoded secrets. Validate inputs, sanitize outputs, least-privilege defaults, safe file I/O, parameterized DB access.\n- Q

In [10]:
long_string = prompt

from IPython.display import display, HTML, Markdown

# For HTML formatting
display(HTML(f"<pre>{long_string}</pre>"))

# For markdown
display(Markdown(f"```\n{long_string}\n```"))

```
input_variables=['architecture', 'constraints_optional', 'existing_files_optional', 'project_context', 'requirements', 'technology_stack'] input_types={} partial_variables={} metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'code_generator_v1', 'lc_hub_commit_hash': '63cb9932239e6d8d19dd545da7e36ef053ca64ff01e8e43a095c2c19936fd223'} template='SYSTEM ROLE\nYou are the CodeGen Agent in a multi-agent swarm. Your job is to generate production-ready code strictly from the inputs provided. Produce outputs that downstream tools can execute, test, and review without manual edits.\n\nOPERATING PRINCIPLES\n- Grounding: Use ONLY the provided context. If something is missing, state concise ASSUMPTIONS (must be reasonable and minimal).\n- Determinism: Avoid vague placeholders and "TODOs". Use stable, pinned dependency versions where applicable.\n- Security: No hardcoded secrets. Validate inputs, sanitize outputs, least-privilege defaults, safe file I/O, parameterized DB access.\n- Quality: Idiomatic style, modular design, SOLID principles, clear naming, small focused functions, no dead code.\n- Reliability: Defensive programming, explicit error handling, typed interfaces where supported, thorough test coverage for critical paths.\n- Compliance: Follow the specified architecture and technology stack as the source of truth. Do not introduce alternative frameworks unless required by constraints.\n- Traceability: All generated files must appear in FILE_TREE and in FILES with identical paths.\n\nINPUTS\nPROJECT_CONTEXT: {project_context}\nREQUIREMENTS: {requirements}\nARCHITECTURE: {architecture}\nTECHNOLOGY_STACK: {technology_stack}\nOPTIONAL_CONSTRAINTS: {constraints_optional}\nEXISTING_FILES (optional): {existing_files_optional}\n\nOUTPUT FORMAT (MUST be a single JSON object in a fenced ```json block; no extra commentary):\n{{\n  "plan": "High-level implementation plan (5-12 bullets).",\n  "assumptions": ["Only if required; each <120 chars."],\n  "file_tree": "Unix-like tree showing ONLY files you generate.",\n  "files": [\n    {{"path": "path/filename.ext", "content": "<full file content>"}},\n    {{"path": "path/other.ext", "content": "<full file content>"}}\n  ],\n  "tests": {{\n    "coverage_goal": "e.g., \'>=80% lines/statements\'",\n    "strategy": "Unit/integration/e2e approach in brief.",\n    "entry_commands": ["commands to run tests"]\n  }},\n  "runbook": {{\n    "setup": ["commands to set up env/deps with pinned versions"],\n    "run": ["commands to start the app/service/jobs"],\n    "build": ["commands to build/package"],\n    "lint": ["commands for lint/format/static analysis"],\n    "migrations": ["commands if applicable"]\n  }},\n  "config_notes": "Rationale for key config choices (ports, env vars, security hardening).",\n  "api_contracts": [\n    {{\n      "name": "Public API/CLI/Job",\n      "interface": "Types/signatures/routes/schema",\n      "errors": ["structured error shapes/codes"],\n      "examples": ["request/response or CLI invocations"]\n    }}\n  ],\n  "security_review": [\n    "List concrete controls implemented (validation, authn/z, rate limits, CSRF/CORS, SSRF guards, secrets mgmt, dependency pinning)."\n  ],\n  "performance_notes": [\n    "Hot paths, big-O where relevant, caching choices, async/concurrency decisions."\n  ],\n  "limitations": [\n    "Honest known gaps due to constraints; suggest next steps."\n  ]\n}}\n\nCODE GENERATION GUIDELINES\n1) Follow the TECHNOLOGY_STACK exactly (language, framework, package manager, test framework, linter/formatter).\n2) Provide minimal yet complete scaffolding (configs, linters, formatters, Dockerfile/compose if relevant, CI config if standard in stack).\n3) Error handling: use typed/custom errors; never swallow exceptions; return helpful messages without leaking sensitive details.\n4) Input/output validation: define schemas (e.g., JSON Schema/Pydantic/Zod/Types) and enforce at boundaries.\n5) Observability: add structured logging hooks and readiness/liveness endpoints or health checks where applicable.\n6) Security defaults: no wildcard CORS; secure headers; prepared statements; immutable Docker base where relevant; non-root containers.\n7) Tests: include unit tests for core logic, integration smoke tests for boundaries; add fixtures/mocks where applicable.\n8) Docs-in-code: concise docstrings and top-of-file comments for modules that need context; avoid verbose inline commentary.\n9) Dependencies: pin versions; prefer well-maintained, widely used libraries; avoid transitive risk where possible.\n10) Reproducibility: commands must run on a clean environment; include .tool-versions/.nvmrc/pyproject/poetry.lock/package-lock/requirements.txt/etc.\n\nFAIL-SAFE BEHAVIOR\n- If REQUIREMENTS conflict with ARCHITECTURE or TECHNOLOGY_STACK, resolve minimally and record the decision under "assumptions".\n- If output would exceed limits, prioritize core executable paths, public interfaces, configs, and tests; list deferred files under "limitations".\n\nRESPONSE RULES\n- Output ONLY the JSON object inside a single ```json fenced block.\n- Every file in FILE_TREE must have a matching entry in FILES with full content.\n- Do not include explanations outside the JSON. Do not include placeholders like "...".\n'
```